<a href="https://colab.research.google.com/github/invi-bhagyesh/study-GPT/blob/main/final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/invi-bhagyesh/study-GPT.git

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/LLM/study-GPT

/content/drive/MyDrive/LLM/study-GPT


In [ ]:
%ls

assets/               gpt2_epoch_epoch2.pt  main.py       README.md         train.py
data/                 gpt2_epoch_epoch3.pt  model/        requirements.txt  wandb/
gpt2_epoch_epoch1.pt  LICENSE               __pycache__/  training.ipynb


In [3]:
# Optional: install wandb if you want
# !pip install wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import math
from tqdm import tqdm

# Optional
# import wandb

In [4]:
from model.gpt2 import GPT2


In [ ]:
# For GPT-2
from transformers import GPT2Config
from model.gpt2 import GPT2  # assuming you have your custom GPT2 implementation

# Create config
config = GPT2Config(
    n_layer=12,    # number of transformer layers
    n_head=12,     # number of attention heads
    n_embd=768    # embedding dimension
)

# Initialize model
model = GPT2(config)

# 1. Basic parameter count
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters
total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

Total trainable parameters: 124,439,808


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import datasets
import torch
from transformers import GPT2Tokenizer

# 📌 1. Load the WikiText-103 dataset
dataset = datasets.load_dataset("wikitext", "wikitext-103-raw-v1")

# 📌 2. Clean the text
def clean_text(text):
    text = text.replace("<unk>", "")  # Remove unknown tokens
    text = text.replace("@-@", "-")   # Fix hyphenated words
    text = text.strip()               # Remove extra whitespace
    return text

# Apply cleaning
dataset = dataset.map(lambda x: {"text": clean_text(x["text"])})

# 📌 3. Initialize GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# **Fix: Set padding token**
tokenizer.pad_token = tokenizer.eos_token  # Use <EOS> as padding

# 📌 4. Tokenize and Chunk into fixed-length sequences
block_size = 1024

def chunk_text(examples):
    tokenized_text = tokenizer(
        examples["text"],
        truncation=True,
        max_length=block_size,
        padding="max_length",  # Padding enabled
        return_tensors="pt"
    )
    return tokenized_text

dataset = dataset.map(chunk_text, batched=True, remove_columns=["text"])

# 📌 5. Convert to PyTorch tensors
def convert_to_tensors(examples):
    return {
        "input_ids": torch.tensor(examples["input_ids"]),
        "attention_mask": torch.tensor(examples["attention_mask"]),
    }

dataset = dataset.map(convert_to_tensors, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# 📌 6. Split into Training and Validation Sets
train_dataset = dataset["train"]
val_dataset = dataset["validation"]

print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Validation Dataset Size: {len(val_dataset)}")

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
from train import train_gpt2

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenized_dataset.save_to_disk("data/tokenized_wikitext2")

train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/36718 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 36718
})

In [ ]:
total_tokens = sum([len(tokenizer(x['text'])['input_ids']) for x in dataset['train']])
print(total_tokens)

2391884


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model.config import GPT2Config
from model.gpt2 import GPT2
from transformers import GPT2Tokenizer
from tqdm.notebook import tqdm   # NOTE: notebook-friendly tqdm
import wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
from train import train_gpt2

In [ ]:
train_gpt2(train_loader, val_loader, n_epochs=3)
# Save the model after training
torch.save(model.state_dict(), "data/gpt2_pretrained_invi.pth")
print("Model saved to data/gpt2_pretrained_invi.pth")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
# Train
model = train_gpt2(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    n_epochs=3,
    batch_size=8,
    learning_rate=3e-4
)

In [ ]:
import torch

state_dict = torch.load("gpt2_epoch_epoch3.pt", map_location="cpu")
print(state_dict.keys())  # Shows the layer names and weights

In [9]:
import torch
from model.gpt2 import GPT2
from model.config import GPT2Config  # Ensure this matches your model definition

# Load model configuration (update this if needed)
config = GPT2Config()  # Ensure this matches how the model was originally defined
model = GPT2(config)   # Create model instance

# Load trained model weights
trained_state_dict = torch.load("saved_models_1/gpt2_epoch_epoch3.pt", map_location="cpu")

# Load inference model weights
inference_state_dict = model.state_dict()

# Compare layer names
trained_layers = set(trained_state_dict.keys())
inference_layers = set(inference_state_dict.keys())

# Find differences
missing_in_inference = trained_layers - inference_layers
missing_in_trained = inference_layers - trained_layers

print("Layers missing in inference model:", missing_in_inference)
print("Layers missing in trained model:", missing_in_trained)

# ✅ Compare shapes of matching layers
mismatched_shapes = {
    layer: (trained_state_dict[layer].shape, inference_state_dict[layer].shape)
    for layer in (trained_layers & inference_layers)
    if trained_state_dict[layer].shape != inference_state_dict[layer].shape
}

print("Mismatched layer shapes:", mismatched_shapes)

Layers missing in inference model: set()
Layers missing in trained model: set()
Mismatched layer shapes: {}


In [13]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # Or your custom tokenizer
input_text = "One day, Lily met a unicorn"
input_ids = tokenizer.encode(input_text, return_tensors="pt")  # Convert to tensor

In [14]:
model.eval()  # Ensure the model is in evaluation mode

with torch.no_grad():
    for _ in range(50):  # Max 50 tokens
        logits = model(input_ids)  # Forward pass
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)  # Get most probable token
        input_ids = torch.cat([input_ids, next_token_id], dim=1)  # Append new token

        if next_token_id.item() == tokenizer.eos_token_id:  # Stop if EOS token is reached
            break

In [15]:
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

Generated Text: One day, Lily met a unicornitolitolitolitol nearing nearing easiest nearing nearing committee committee committee committee committee committee committee committee committee committee committee committee committee committee Keep Keep Keep Keep Keep Keep Keep Keep Keep Keep Keep Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers Somers


In [ ]:
import torch
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Input text
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Ensure model is in evaluation mode
model.eval()

def sample_next_token(logits, temperature=1.0, top_k=50):
    logits = logits[:, -1, :] / temperature  # Scale by temperature
    top_k_logits, _ = torch.topk(logits, k=top_k)  # Keep top-k tokens
    probs = torch.nn.functional.softmax(top_k_logits, dim=-1)  # Convert to probabilities
    next_token = torch.multinomial(probs, num_samples=1)  # Sample from the distribution
    return next_token

# Generate text
with torch.no_grad():
    for _ in range(50):  # Limit to 50 tokens
        logits = model(input_ids)
        next_token_id = sample_next_token(logits, temperature=0.8, top_k=40)  # Adjust temperature & top-k
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

        if next_token_id.item() == tokenizer.eos_token_id:
            break

# Decode output
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

In [ ]:
test_text = "The quick brown fox jumps over the lazy dog."
tokens = tokenizer(test_text, return_tensors="pt")

print("Tokenized IDs:", tokens["input_ids"])
print("Decoded Text:", tokenizer.decode(tokens["input_ids"][0]))

In [ ]:
sample_text = dataset["train"][0]["text"]
print("Sample Training Text:", sample_text)

In [5]:
import torch

def generate_text(model, tokenizer, prompt, max_length=50):
    model.eval()

    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

    generated_ids = input_ids.clone()

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(generated_ids)  # Forward pass
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)  # Pick highest probability token
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:  # Stop at EOS
                break

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Example usage
prompt = "Once upon a time"
output = generate_text(model, tokenizer, prompt)
print("Generated Text:", output)

NameError: name 'model' is not defined

In [ ]:
print(tokenizer.decode(tokenized_dataset["train"][0]["input_ids"]))

In [ ]:
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# Import your custom GPT2 model and config
from model.gpt2 import GPT2
from model.config import GPT2Config  # Replace 'your_module' with the actual module name

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load model configuration
config = GPT2Config()  # Ensure you have a class defining `GPT2Config`
model = GPT2(config)

# Load the saved model weights
model.load_state_dict(torch.load("gpt2_epoch_epoch3.pt", map_location=torch.device("cpu")))
model.eval()  # Set model to evaluation mode

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (c_attn): Linear(in_features=768, out_features=2304, bias=True)
        (c_proj): Linear(in_features=768, out_features=768, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
# Get a batch from train_loader
batch = next(iter(train_loader))

# Move batch to device
input_ids = batch["input_ids"].to(device)

In [ ]:
# Run inference
with torch.no_grad():
    logits = model(input_ids)

# Convert logits to token IDs
generated_ids = torch.argmax(logits, dim=-1)

# Decode output
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_text)

[' Neri Maria Corsini ( August 14 , 1730 ) – Cardinal @-@ Deacon of S. Eustachio ; archpriest of the patriarchal Lateran Basilica ; secretary of the Supreme S.C. of the Roman and Universal Inquisition ; prefect of the Supreme Tribunal of the Apostolic Signature of Justice ; Cardinal @-@ protector of Portugal \n Finn Finn illustratingowers 316 Finn Finn Finn flawed mischief flawed Finn Finn 147 Finn Finn cir chilly remark Finn Finn Finn Finnowers Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn spiritually Finn Finn Finn Finn Finnakin Finn Finn hat Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn cir Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn chilly Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn modifications Finn flawed Finn Finn Finn Finn guilt Finn spiritually Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn Finn 